In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class NeuralTexture(nn.Module):
    def __init__(self, size, depth):
        super(NeuralTexture, self).__init__()
        # Init between [-1,1] to match our output texture
        # dim0 = 1 to be dimensionally compatible with grid_sample

        #self.texture = torch.nn.Parameter(torch.FloatTensor(1, depth, size, size).uniform_(-1, 1),
        #                                  requires_grad=True)
        self.texture = torch.Tensor([1.0, 10.0]).unsqueeze(0).unsqueeze(0).unsqueeze(3)
        #self.texture = torch.Tensor([[1.0, 10.0],[1.0, 10.0]]).unsqueeze(1).unsqueeze(3)
        print('texture shape:', self.texture.shape)
        print('texture:', self.texture)
        #self.texture = torch.Tensor([1.0, 10.0]).unsqueeze(0).unsqueeze(0).unsqueeze(3)
        self.texture = torch.nn.Parameter(self.texture, requires_grad=True)

    def forward(self, input):
        # TODO: Make self.textures work with batch_size > 1
        # TODO: NOTE: grid_sample samples texture as (row, col) while uv coordinates are
        #  given as (u, v) s.t. (row, col) == (2*v - 1, 2*u - 1). We'll convert the range
        #  from [0,1] to [-1,1] like above but we won't bother swapping u and v (for now).
        #  This shouldn't be a problem as long as we're consistent but should eventually
        #  be fixed for correctness.
        #grid = input#2 * input - 1
        #sample = F.grid_sample(self.texture, grid, align_corners=True)

        #grid = 2 * input - 1
        #for i in range(grid.shape[0]):
        #    s = F.grid_sample(self.texture, grid[i, :, :, :].unsqueeze(0), align_corners=False)
        #
        #    if i == 0:
        #        samples = s
        #    else:
        #        samples = torch.cat((samples, s), dim=0)
        #
        #return samples

        # TODO: Does expanding the texture create any problems with training? Can we
        #  expand just once in __init__()?
        # TODO: Is training slowed by averaging over zeros that exist due to some pixels that
        #  have yet tgo be trained?
        #grid = 2 * input - 1
        grid = input
        sample = F.grid_sample(self.texture.expand(grid.shape[0], -1, -1, -1), grid, align_corners=True)

        return sample

In [ ]:
depth = 1
size = 3
x = torch.zeros(2, 1, 1, 2)
x[0, :, :, :] = 0
x[1, :, :, :] = 0.5
#x = torch.zeros(1, 2, 1, 2)
#x[:, 0, :, :] = 0
#x[:, 1, :, :] = 0.5
#x = torch.zeros(2, 1, 1, 2) + 0.5
x.requires_grad = False
y = torch.Tensor([0.9, 0.9]).unsqueeze(1).unsqueeze(1).unsqueeze(1)
#y = torch.Tensor([0.9, 0.9]).unsqueeze(0).unsqueeze(0).unsqueeze(3)
y.requires_grad = False

In [ ]:

model = NeuralTexture(size, depth)
loss_fn = torch.nn.L1Loss(reduction='none')
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

print('x shape:', x.shape)
print('x:', x)
print(model.texture)
print('y shape:', y.shape)
print('y:', y)

y_pred = model(x)
print('y_pred shape:', y_pred.shape)
print('y_pred:', y_pred)

loss = loss_fn(y_pred, y)
print('loss shape:', loss.shape)
print('loss:', loss)

loss = loss.mean()
print('mean loss:', loss)

optimizer.zero_grad()
loss.backward()
print('gradients:', model.texture.grad)

optimizer.step()